In [3]:
import numpy as np
import pandas as pd
import GPy
from scipy.special import expit
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from scipy.spatial import distance
import statsmodels.api as sm
import warnings
import numpy as np
from scipy.optimize import differential_evolution
from scipy.stats import rankdata
from sklearn.cross_decomposition import CCA
import math
import copy
from math import log
from collections import Counter
import pickle
from sklearn.linear_model import Lasso
warnings.filterwarnings("ignore")

In [9]:
with open('relation_matrix_dic.pkl', 'rb') as f:
    relation_matrix_dic = pickle.load(f)

error_array = np.load('error.npy')

# define a new adjacency_matrix_dic: adjacency_matrix_dic[i] = relation_matrix_dic[i].iloc[0:20, 0:20]
adjacency_matrix_dic = {i: relation_matrix_dic[i].iloc[0:5, 0:5] for i in relation_matrix_dic}

adjacency_matrix = adjacency_matrix_dic[31]

In [11]:
adjacency_matrix = np.array(adjacency_matrix)

In [12]:
adjacency_matrix

array([[0., 1., 1., 1., 1.],
       [1., 0., 0., 1., 1.],
       [1., 0., 0., 0., 0.],
       [1., 1., 0., 0., 1.],
       [1., 1., 0., 1., 0.]])

In [13]:
# Get the upper triangular portion (excluding diagonal)
upper_triangular = np.triu(adjacency_matrix, k=1)

# Count the number of 1s
num_ones = np.sum(upper_triangular)

In [17]:
K = 2
length_beta = 1 + K * len(adjacency_matrix) + K**2 * num_ones

In [26]:
length_beta = int(length_beta)
length_beta 

39

In [34]:
beta_star = np.zeros(length_beta)

# Randomly select 10 indices to have non-zero values
non_zero_indices = np.random.choice(length_beta, size=5, replace=False)

# Assign values from normal distributions to the selected indices
for idx in non_zero_indices:
    mean = np.random.randint(-5, 15)  # Random integer mean in the range [-10, 10]
    beta_star[idx] = np.random.normal(loc=mean, scale=2)  # Normal distribution with the selected mean and std dev = 1

print(beta_star)

[ 0.          7.28061152  0.          0.          0.          0.
  0.          0.          2.86269357  0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          4.50931371  0.          0.         -1.22792887
  0.          0.          0.          0.          7.04500862  0.
  0.          0.          0.        ]


In [38]:
np.save('adjacency_matrix.npy', adjacency_matrix)

In [35]:
# Save the array as 'beta_star.npy'
np.save('beta_star.npy', beta_star)

print("Array saved as 'beta_star.npy'.")

Array saved as 'beta_star.npy'.


# test_transform X

In [41]:
# Generate A: Random integers from 1 to K
A = np.random.randint(1, K + 1, size=len(adjacency_matrix))

# Extract upper triangular part (excluding diagonal)
upper_triangular = np.triu(adjacency_matrix, k=1)

# Generate B based on the rules
B = []
for i in range(len(adjacency_matrix)):
    for j in range(len(adjacency_matrix)):
        if upper_triangular[i, j] == 1:
            B.append(K * (A[i] - 1) + A[j])

B = np.array(B)

# Create X
# Step 1: Start with 1
X = [1]

# Step 2: Add one-hot encoding of A (length K for each element in A)
for a in A:
    one_hot_a = np.eye(K)[a - 1]  # One-hot encoding
    X.extend(one_hot_a)

# Step 3: Add one-hot encoding of B (length K^2 for each element in B)
for b in B:
    one_hot_b = np.eye(K**2)[b - 1]  # One-hot encoding
    X.extend(one_hot_b)

X = np.array(X)

# Output
print("A:", A)
print("B:", B)
print("X:", X)


A: [1 2 2 2 2]
B: [2 2 2 2 4 4 4]
X: [1. 1. 0. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0.
 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1.]


In [42]:
len(X)

39

In [43]:
np.dot(X, beta_star)

17.18831371630942